In [ ]:
!pip install transformers
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import csv 
import xml.etree.ElementTree as ET
import fileinput 
import re
import os

minConfidence = 0.9
sentimentModel = pipeline("sentiment-analysis", model = "Seethal/sentiment_analysis_generic_dataset")

#make data struct based on import - dictionary of authors holding book objects
#book objects -> dictionary of textpart_names to textpart objects
#textpart objects -> hold the original text, original sentiment, english text (if exists), average english sentiment from each sentence, author, and book
class Textpart:
  textpart = ""
  text = ""
  author = ""
  book_title = ""
  engText = ""
  engSent = 0.0
  originalSent = 0.0
  
  def __init__(self, _textpart, _text, _title, _author, _engText):
    self.textpart = _textpart
    self.text = _text
    self.author = _author
    self.book_title = _title
    self.engText = _engText
    if _engText != "none":
      #seperate text into sentences and run sentiment on them
      sentences = _engText.split(".")
      total = 0
      sentencesProcessed = 0
      for s in sentences:
        s+= "."
        temp = sentimentModel(s)
        if temp[0]["score"] > minConfidence:
          sentencesProcessed+=1
          if(temp[0]["label"] == "LABEL_2"):
            total+=1
          elif(temp[0]["label"] == "LABEL_0"):
            total-=1
      self.engSent = total/sentencesProcessed
    #code for original text sentiment

class Book:
  #dictionary is textpartName, TextPart
  author = ""
  title = ""
  parts = {}
  def __init__(self, textPartObj):
    self.parts[textPartObj.textpart] = textPartObj
    self.title = textPartObj.book_title
    self.author = textPartObj.author

  def addTextPart(self, textPartObj):
    if(textPartObj.book_title != self.title):
      print("error in book datastruct")
    else:
      self.parts[textPartObj.textpart] = textPartObj

class Author:
  name = "" 
  books = {}
  def __init__(self, bookObj):
    self.name = bookObj.author
    self.books[bookObj.title] = bookObj
  def addBook(self, bookObj):
    if(bookObj.author != self.name):
      print("error in Author datastruct")
    else:
      self.books[bookObj.title] = bookObj

#effictively a dictionary of authors with the setup stuff
class Texts: 
  authors = {}
  def __init__(self):
    authors = {}
  def addTextPart(self, txtprt, text, title, author, engText):    
    temp = Textpart(txtprt, text, title, author, engText)
    if len(self.authors)== 0:
      book = Book(temp)
      writer = Author(book)
      self.authors[writer.name] = writer
    else:
      authorFound = False;
      for a in self.authors:
        #check if there is an author that matches
        if(a.title == author):
          authorFound = True
          titleFound = False
          for t in a.books:
            #check if there is a book with same title
            if t == title:
              titleFound = True
              t.addTextPart(temp)
          if not titleFound:
            book = Book(temp)
            a.addBook(book)
      if not authorFound:
        book = Book(temp)
        writer = Author(book)
        self.authors[writer.name] = writer




#set csvInName to the path to the csv, and delim to the delimiter used 
csvInName = "/Input/csvTextPartsWithTitleAuthorAndEng.csv"
csvOutName = "/output/storedDatastruct.csv"
delim = ","
textpart = ""
text = ""
author = ""
title = ""
engText = ""
holder = Texts()
if (csvInName != ""):
  #input csv schema: textpart, text, title, author, english text
  csvFile = open(csvInName, "r")
  csv_reader = csv.reader(csvFile, delimiter = delim)
  firstLine = True
  for row in csv_reader:
    if not firstLine:
      #make the object baby 
      textpart = row[0]
      text = row[1]
      title = row[2]
      author = row[3]
      engText = row[4]
      holder.addTextPart(textpart, text, title, author, engText)
    else:
      firstLine = False


print("Job done!")
    
#label 0 is negative
#label 1 is neutral
#label 2 is positive
#for multiple calls, use a list





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


AttributeError: ignored

In [ ]:
inWords = open("Input/words.txt", "r")
outWords = open("output/words.txt", "w")





In [ ]:
!pip install transformers
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
sentiment = pipeline("sentiment-analysis", model = "Seethal/sentiment_analysis_generic_dataset")

print(sentiment("Senate Woot Woot!"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


For making the latin sentiment analysis:
  Scrape wiktionary for latin - use pd
  Use template from nitai's github to make the trainer - 
  Make dataset using template on nitai's github 